In [1]:
from parameter_graph import ParameterGraph

* We will use `n` qubits to simulate the interactions between `n` entities in a game.
* Each entity is described by at most three variables, each of which takes a value from $+1$ to $-1$.
* The variables are called `X`, `Y` and `Z`, and must satisfy $X^2 + Y^2 + Z^2 \leq 1$.
* Note that this implies that an extreme value ($+1$ or $-1$) for any variable implies that the others must be $0$.
* The value of $X^2 + Y^2 + Z^2$ is determined by how the entities interact, and cannot be directly set to a given value.
* A further set of variables will describe relationships between pairs of entiities.
* These are called `XX`, `XY`, `XZ`, `YX`, etc.
* The `XX` variable takes the value $+1$ when $X=+1$ for both entities, or $X=-1$ for both entities. It takes the value $-1$ when $X=+1$ for one entity and $X=-1$ for the other.
* Similarly for the rest.

### An example

We'll look at an example where the entities are island nations, and we'll have `n=4` of them. The names of the entities are given in the list `labels`.

In [2]:
labels = ['Altstetten','Brugg','Frick','Rheinfelden']

The dictionary `variables` is then used to determine what the `X`, `Y` and `Z` variables are used for. We'll use only two, and have them represent the aggression and competence of the government for each nation.

The idea behind this choice is that a government that pursues only war or only peace will sometimes be acting in its nation's best interest, and sometimes not. So it cannot be regarded as fully competent or incompetent. A competent government would seek war or peace only when it is in the best interests of the nation, and have no bias towards either. An incompetent government would pursue war and peace randomly, and also have no bias toward either. A government that prefers war would often be incomptent.

You might not fully agree with this assessment (I'm not sure I do myself), but it does capture the required relation between the variables.

In [3]:
variables={}
variables['X']  = 'Aggression'
variables['Z']  = 'Competence'

Given these choice, what does `XX` represent? It is something that will be true about the relationship for two very aggresive nations, as well as two very peaceful ones. And the opposite will be true of the relationship between an aggressive nation and a peaceful one.

We will take this to be the amount that the citizens of the nations engage with each other. The citizens of two aggressive nations will fight whenever they see each other. The citizens of two peaceful nations will trade. In both cases there is strong engagement (though it takes different forms).

In [4]:
variables['XX'] = 'Engagement between citizens'

Now what is true for two nations that are either both competent, or both incompetent? Or, alternatively, what is not true for the relationship between a competent nation and an incompetent one?

We'll take this to be the amount that they collaborate on shared goals.

In [5]:
variables['ZZ'] = 'Collaboration between governments'

Now for the variables that are not symmetric. These desribe a quality of the relationship between given the competence of one nation and the agression of the other.

We'll take this to be the amount that the policy of one government (for which we are considering the competence) focusses on the other nation (for which we are considering the aggression). The idea here is that a competent government would take care to defend against or appease an aggressive neighbour. Whereas a competent government would probably be too busy trying to make money on trade deals with a peaceful neighbour.

These asymettric variables are specified by tuples, so that the names of the islands can be filled in.

In [6]:
variables['XZ'] = ('Focus on','by')
variables['ZX'] = ('Focus of','on')

Now we have enough information to initialize the network of nations.

In [7]:
n = 4
network = ParameterGraph(n,labels=labels,variables=variables)

The initialization gives the $Z=1$ state for all entities by default. So all nations start off completely competent.

We can use `set_state()` to change this. Note that the values given for the variables need not satisfy any particular value for $X^2 + Y^2 + Z^2$, since `set_state()` will automatically renormalize to the required value.

In [8]:
network.set_state({'Aggression':1,'Competence':0},'Altstetten')
network.set_state({'Aggression':0,'Competence':1},'Brugg')
network.set_state({'Aggression':0.5,'Competence':0.5},'Frick')
network.set_state({'Aggression':-1,'Competence':0},'Rheinfelden')

We can inspect the states at any time using `get_state`.

In [9]:
def print_states():
    for island in labels:
        print(island)
        state = network.get_state(island)
        for variable in state:
            print('   ',variable+':',state[variable])
        print('')
        
print_states()

Altstetten
    Aggression: 1.0
    Competence: -0.00244140625

Brugg
    Aggression: 0.005777994791666667
    Competence: 1.0

Frick
    Aggression: 0.7069498697916666
    Competence: 0.698486328125

Rheinfelden
    Aggression: -1.0
    Competence: -0.0068359375



We can also look at the relationships.

In [10]:
def print_relationships():
    for j, island1 in enumerate(labels):
        for k, island2 in enumerate(labels):
            if k>j:
                print(island1,'and',island2)
                relationship = network.get_relationship(island1,island2)
                for variable in relationship:
                    print('   ',variable+':',relationship[variable])
                print('')
            
print_relationships()

Altstetten and Brugg
    Engagement between citizens: 0.019775390625
    Focus on Altstetten by Brugg: 1.0
    Focus of Altstetten on Brugg: -0.0048828125
    Collaboration between governments: -0.00244140625

Altstetten and Frick
    Engagement between citizens: 0.713134765625
    Focus on Altstetten by Frick: 0.700439453125
    Focus of Altstetten on Frick: 0.00390625
    Collaboration between governments: -0.000732421875

Altstetten and Rheinfelden
    Engagement between citizens: -1.0
    Focus on Altstetten by Rheinfelden: -0.001953125
    Focus of Altstetten on Rheinfelden: 0.012939453125
    Collaboration between governments: 0.01611328125

Brugg and Frick
    Engagement between citizens: 0.00537109375
    Focus on Brugg by Frick: 0.010986328125
    Focus of Brugg on Frick: 0.708251953125
    Collaboration between governments: 0.698486328125

Brugg and Rheinfelden
    Engagement between citizens: -0.019775390625
    Focus on Brugg by Rheinfelden: -0.00927734375
    Focus of Brug

Here we see that both Brugg and Frick have their eyes on the very aggressive Altstetten. We also see that Brugg and Frick are both very willing to collaborate. The next chapter in the history in these islands is therefore clear: Brugg and Frick are going to try to reduce the threat from Alstetten! Both attack in an attempt to place a competent government in charge of Altstetten.

This is done using `set_state()` again. But this time it is a conditional operation, the effectiveness of which depends on how aggressive the attackers are. If they had maximum aggression, the `set_state` command would act exactly as it would when applied normally. For minimal aggression, it would have no effect. Otherwise, the effect is somewhere between the two, and will also cause a change in the relationshop between the two nations.

In [11]:
network.set_state({'Aggression':0,'Competence':1},'Altstetten',q_if=['Aggression','+','Brugg'])
network.set_state({'Aggression':0,'Competence':1},'Altstetten',q_if=['Aggression','+','Frick'])

Let's see what the state of the islands are after this war.

In [14]:
print_states()

Altstetten
    Aggression: 0.054280598958333336
    Competence: 0.69189453125

Brugg
    Aggression: 0.009847005208333334
    Competence: 0.721923828125

Frick
    Aggression: 0.712646484375
    Competence: 0.643310546875

Rheinfelden
    Aggression: -1.0
    Competence: -0.0029296875



Altstetten has indeed become a far less aggressive and more competent nation. Brugg's competence seems to have been reduced a bit as a consequence of the war. Frick seems to be relatively unscathed.

Now let's look at the relationships.

In [16]:
print_relationships()

Altstetten and Brugg
    Engagement between citizens: -0.679931640625
    Focus on Altstetten by Brugg: 0.0927734375
    Focus of Altstetten on Brugg: 0.068603515625
    Collaboration between governments: 0.952392578125

Altstetten and Frick
    Engagement between citizens: -0.07470703125
    Focus on Altstetten by Frick: 0.169677734375
    Focus of Altstetten on Frick: 0.53564453125
    Collaboration between governments: 0.467041015625

Altstetten and Rheinfelden
    Engagement between citizens: -0.07421875
    Focus on Altstetten by Rheinfelden: -0.015625
    Focus of Altstetten on Rheinfelden: -0.68505859375
    Collaboration between governments: 0.0009765625

Brugg and Frick
    Engagement between citizens: -0.003173828125
    Focus on Brugg by Frick: 0.02685546875
    Focus of Brugg on Frick: 0.503662109375
    Collaboration between governments: 0.4677734375

Brugg and Rheinfelden
    Engagement between citizens: 0.010498046875
    Focus on Brugg by Rheinfelden: -0.009521484375
  

It seems that the new government of Altstetten wants to be best buddies with Brugg. It also has a fairly strong desire to collaborate with Frick. The old allies Brugg and Frick are also on fairly good terms. There is now peace between these three.

Rheinfelden is left out of the party, but seems to have no desire to get involved anyway. As a very non-aggressive nation, the other islands are happy to leave it alone.

The world seems pretty stable for now, so that seems to conclude the demo.